In [1]:
def preve(comex_df_NCM,nick_list,date_pred,comp_min,comp_max,md_c,md_r,quanti_col):

    # Confere comprimento mínimo
    if pd.to_datetime(str(date_pred.year-comp_min)+'/'+str(date_pred.month)+'/1') >= comex_df_NCM.date.min():
        
        ############### Limita o comprimento máximo ##############################################
        ad_y=0
        if comp_max!=None:
            month_ini=date_pred.month+1
            if month_ini>12:
                month_ini=1
                ad_y=1
            data_inicio=pd.to_datetime(str(date_pred.year-comp_max+ad_y)+'/'+str(month_ini)+'/1')
            comex_df_NCM=comex_df_NCM[comex_df_NCM.date>=data_inicio]

        
        ################## Produz a Matriz Esparsa marcando as ocorrências ###########
        %run '~/git/BD/Comexstat/function/matriz_esparsa.ipynb'
        comex_df_NCM=matriz_esparsa(comex_df_NCM, quanti_col)        
        ##############################################################################
        
        
        # Parametriza os algoritmos
        RFC=RFC_(max_depth=md_c,n_estimators=1000)
        RFR=RFR_(max_depth=md_r)
        
        # Adequa medidas de tempo para os algoritmos
        comex_df_NCM.date=comex_df_NCM.date.astype(int)
        comex_df_NCM.drop(columns='CO_ANO',inplace=True)

        # Codifica categóricos
        imp_fert_dumm=pd.get_dummies(comex_df_NCM,prefix_sep='~')

        # Separa treino e teste
        train=imp_fert_dumm[imp_fert_dumm.date< imp_fert_dumm.date.max()]
        test =imp_fert_dumm[imp_fert_dumm.date==imp_fert_dumm.date.max()]
       
        

        # Inicia df para coletar resultados de classificação e regressão do loop
        q=pd.DataFrame()
        r=pd.DataFrame()

        # inicia loop de repetição (para teste de variação dos algorítmos randômicos)
        for x in range(rept):

            # Identifica X e y classifier
            X_train=train.drop(columns=[quanti_col,'oc'])
            y_train=train.oc
            X_test =test.drop(columns=[quanti_col,'oc'])
            y_test =test.oc
        
            # ******* CLASSIFCAÇÃO *******
            
            # Fit classifier
            RFC.fit(X_train,y_train)
            
            # Predict classifier
            test_pred=X_test.assign(oc_pred=RFC.predict(X_test)).assign(oc=y_test)\
            .assign(nick_list=str(nick_list))\
            .assign(date_pred=date_pred)\
            .assign(rept=x)
            
            # Armazena
            q=pd.concat([q,test_pred])

            
            # ******* REGRESSÃO *******
            
            # Filtra deixando passar registros ocorridos
            train=train[train.oc==1]
            test=test[test.oc==1]
            
            # verifica o comprimento mínimo
            if (len(train)>0)&(len(test)>0):
                
                # Identifica X e y
                X_train=train.drop(columns=[quanti_col,'oc'])
                y_train=train.KG_LIQUIDO
                X_test =test.drop(columns=[quanti_col,'oc'])
                y_test =test.KG_LIQUIDO
                
                # Fit regressor
                RFR.fit(X_train,y_train)
                
                # Predict classifier
                test_pred=X_test.assign(kg_pred=RFR.predict(X_test)).assign(KG_LIQUIDO=y_test)\
                .assign(nick_list=str(nick_list))\
                .assign(date_pred=date_pred)\
                .assign(rept=x)
                
                # Armazena
                r=pd.concat([r,test_pred])
    
            def f_dumm(df_dumm):
                '''Retorna dumm para original'''
                a=pd.from_dummies(df_dumm.select_dtypes(include='bool'),sep='~')
                b=df_dumm.select_dtypes(exclude='bool')
                c=pd.concat([a,b],axis=1)
                return c
            
        q=f_dumm(q)
        r=f_dumm(r)
        
        score=q.merge(r,how='outer')
    else:
        score=pd.DataFrame()
    return score